#### ORM Mode (aka Arbitrary Class Instances)

Pydantic models can be created from arbitrary class instances to support models that map to ORM objects. To do this:

1. The `Config` property `orm_mode` must be set to `True`.

2. The special constructor `from_orm` must be used to create the model instance.

The example here uses `SQLAlchemy`, but the same approach should work for any ORM.

In [1]:
from typing import Any, List, Dict, Optional
from sqlalchemy import Column, Integer, String, JSON
from sqlalchemy.dialects.postgresql import ARRAY
from sqlalchemy.ext.declarative import declarative_base
from pydantic import BaseModel, Field, constr
from pydantic.utils import GetterDict
from xml.etree.ElementTree import fromstring

In [2]:
Base = declarative_base()

In [3]:
class CompanyOrm(Base):
    __tablename__ = "companies"
    id = Column(Integer, primary_key=True, nullable=False)
    public_key = Column(String(20), index=True, nullable=False, unique=True)
    name = Column(String(63), unique=True)
    domains = Column(ARRAY(String(255)))

In [4]:
class CompanyModel(BaseModel):
    id: int
    public_key: constr(max_length=20)
    name: constr(max_length=63)
    domains: List[constr(max_length=255)]

    class Config:
        orm_mode = True

In [5]:
co_orm = CompanyOrm(
    id=123,
    public_key="foobar",
    name="Testing",
    domains=["example.com", "foobar.com"],
)
print(co_orm)

In [6]:
co_model = CompanyModel.from_orm(co_orm)
print(co_model)

id=123 public_key='foobar' name='Testing' domains=['example.com', 'foobar.com']


##### Reserved names

You may want to name a Column after a reserved SQLAlchemy field. In that case, Field aliases will be convenient.

In [7]:
class MyModel(BaseModel):
    metadata: Dict[str, str] = Field(alias="metadata_")

    class Config:
        orm_mode = True

In [8]:
class SQLModel(Base):
    __tablename__ = "my_table"
    id = Column("id", Integer, primary_key=True)
    metadata_ = Column("metadata", JSON)

> ##### Note
> 
> The example above works because aliases have priority over field names for field population. Accessing `SQLModel`'s `metadata` attribute would lead to a `ValidationError`.

##### Recursive ORM models

ORM instances will be parsed with `from_orm` recursively as well as at the top level. Here a vanilla class is used to demonstrate the principle, but any ORM class could be used instead.

In [9]:
class PetCls:
    def __init__(self, *, name: str, species: str):
        self.name = name
        self.species = species

In [10]:
class PersonCls:
    def __init__(self, *, name: str, age: float = None, pets: List[PetCls]):
        self.name = name
        self.age = age
        self.pets = pets

In [11]:
class Pet(BaseModel):
    name: str
    species: str

    class Config:
        orm_mode = True

In [12]:
class Person(BaseModel):
    name: str
    age: float = None
    pets: List[Pet]

    class Config:
        orm_mode = True

In [13]:
bones = PetCls(name='Bones', species='dog')
orion = PetCls(name='Orion', species='cat')
anna = PersonCls(name='Anna', age=20, pets=[bones, orion])
anna_model = Person.from_orm(anna)
print(anna_model)

name='Anna' age=20.0 pets=[Pet(name='Bones', species='dog'), Pet(name='Orion', species='cat')]


##### Data binding

Arbitrary classes are processed by pydantic using the `GetterDict` class, which attempts to provide a dictionary-like interface to any class. You can customise how this works by setting your own sub-class of `GetterDict` as the value of `Config.getter_dict` (see `config`).

You can also customise class validation using `root_validators` with `pre=True`. In this case your validator function will be passed a `GetterDict` instance which you may copy and modify.

The `GetterDict` instance will be called for each field with a sentinel as a fallback (if no other default value is set). Returning this sentinel means that the field is missing. Any other value will be interpreted as the value of the field.

In [14]:
xmlstring = """
<User Id="2138">
    <FirstName />
    <LoggedIn Value="true" />
</User>
"""

In [15]:
class UserGetter(GetterDict):
    def get(self, key: str, default: Any) -> Any:

        # element attributes
        if key in {'Id', 'Status'}:
            return self._obj.attrib.get(key, default)

        # element children
        else:
            try:
                return self._obj.find(key).attrib['Value']
            except (AttributeError, KeyError):
                return default

In [16]:
class User(BaseModel):
    Id: int
    Status: Optional[str]
    FirstName: Optional[str]
    LastName: Optional[str]
    LoggedIn: bool

    class Config:
        orm_mode = True
        getter_dict = UserGetter

In [17]:
user = User.from_orm(fromstring(xmlstring))
print(user)

Id=2138 Status=None FirstName=None LastName=None LoggedIn=True
